# Загружаем необходимые пакеты и импортируем библиотеки

---

In [ ]:
# устанавливаем библиотеку implicit 
%pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 62.3 MB/s eta 0:00:00


In [2]:
# загружаем библиотеки
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit

# Подготовка и обработка данных

---

In [38]:
# загружаем данные из источника
data = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [39]:
# создаем копию
df = data.copy()

In [40]:
# посмотрим на данные 
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


In [41]:
# и так тоже посмотрим
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   event_date       object
 1   event_timestamp  int64 
 2   vacancy_id_      int64 
 3   cookie_id        object
 4   user_id          object
 5   event_type       object
dtypes: int64(2), object(4)
memory usage: 562.7+ MB


In [42]:
# Дубликатов нет в данных - но на всякий случай
df = df.drop_duplicates()

Попробуем совместить user_id и cookie_id - это первая исследовательская позиция

---

Идея в том, что для каждого куки алгоритм сохранит последнего использующего устройство юзера, и если пользователь один и тот же мы увеличим знания о взаимодействии пользователя с платформой на разных устройстовах, и ничего страшного что у нас на домашний пк могут заходить разные пользователи, авторизация (при доработке сменить последнего юзера), для не авторизированных пользователь программа будет запоминать взаимодействия уникального устройства

In [43]:
df.tail(25) # посмотрим на пустый значения user_id

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
12292563,2022-08-11,1660240768,250327,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response
12292564,2022-08-11,1660240674,133079,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response
12292565,2022-08-11,1660240721,134462,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response
12292566,2022-08-11,1660240655,136817,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response
12292567,2022-08-11,1660239343,164570,67eb24b76f0b47b29b0390fb465c20bc,None,preview_click_response
12292568,2022-08-11,1660239042,164570,67eb24b76f0b47b29b0390fb465c20bc,None,preview_click_response
12292569,2022-08-11,1660242236,110136,ce775682e192406dbb13af8b4fc9d5dc,1a63389451f744bdb14c743c208f9a15,preview_click_response
12292570,2022-08-11,1660242225,135238,ce775682e192406dbb13af8b4fc9d5dc,1a63389451f744bdb14c743c208f9a15,preview_click_response
12292571,2022-08-11,1660243140,143721,b9cdb878e067483b807d50de917acacd,257e6729e1754ff193d1dbdcfb6914d6,preview_click_response
12292572,2022-08-11,1660245131,106505,5b0c4eac6d99495e9c7307f79479f41b,a1c83099275e4ba3ba562ed546d1048e,preview_click_response


In [44]:
# создадим словарик - который разметит новый вариант пользователя который 
new_user_id = dict()
for i, z in zip(df.cookie_id, df.user_id):
  if z == None:
    new_user_id[i] = i
  else:
    new_user_id[i] = z 

In [45]:
# создадим новый вариант пользователя который вбирает в себя 
df['new_user_id'] = df['cookie_id'].map(new_user_id)

In [46]:
df.tail(25) # посмотрим как преобразовалась колонка новый юзер

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,new_user_id
12292563,2022-08-11,1660240768,250327,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response,987aa97723644ef1895aaade85681bb3
12292564,2022-08-11,1660240674,133079,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response,987aa97723644ef1895aaade85681bb3
12292565,2022-08-11,1660240721,134462,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response,987aa97723644ef1895aaade85681bb3
12292566,2022-08-11,1660240655,136817,ae866f07cadf4d5390ee4e10b7e9b091,987aa97723644ef1895aaade85681bb3,preview_click_response,987aa97723644ef1895aaade85681bb3
12292567,2022-08-11,1660239343,164570,67eb24b76f0b47b29b0390fb465c20bc,None,preview_click_response,67eb24b76f0b47b29b0390fb465c20bc
12292568,2022-08-11,1660239042,164570,67eb24b76f0b47b29b0390fb465c20bc,None,preview_click_response,67eb24b76f0b47b29b0390fb465c20bc
12292569,2022-08-11,1660242236,110136,ce775682e192406dbb13af8b4fc9d5dc,1a63389451f744bdb14c743c208f9a15,preview_click_response,1a63389451f744bdb14c743c208f9a15
12292570,2022-08-11,1660242225,135238,ce775682e192406dbb13af8b4fc9d5dc,1a63389451f744bdb14c743c208f9a15,preview_click_response,1a63389451f744bdb14c743c208f9a15
12292571,2022-08-11,1660243140,143721,b9cdb878e067483b807d50de917acacd,257e6729e1754ff193d1dbdcfb6914d6,preview_click_response,257e6729e1754ff193d1dbdcfb6914d6
12292572,2022-08-11,1660245131,106505,5b0c4eac6d99495e9c7307f79479f41b,a1c83099275e4ba3ba562ed546d1048e,preview_click_response,a1c83099275e4ba3ba562ed546d1048e


In [47]:
# удаляем колонки, которые не будут принимать участия в обучении - варинат с объединением пользователя и устройства
df.drop(columns=['event_date', 'event_timestamp', 'user_id', 'cookie_id'], axis=1, inplace=True)

In [48]:
# для удобства немного выравняем данные ста
df = df[['new_user_id', 'vacancy_id_', 'event_type']]

In [49]:
df.head()

,new_user_id,vacancy_id_,event_type
0,951f53de61764ea0b51317200a0dbbfc,129850,show_vacancy
1,f5a2326a17484330aa8cb4019f1b1960,108347,show_vacancy
2,f5a2326a17484330aa8cb4019f1b1960,109069,show_vacancy
3,f5a2326a17484330aa8cb4019f1b1960,171425,show_vacancy
4,f5a2326a17484330aa8cb4019f1b1960,252384,show_vacancy


Блок кода который позволяет агрегировать даннные и удалить лишние взаимодействия (5 раз посмотреть вакансию)

---

In [50]:
df = df.groupby(['new_user_id', 'vacancy_id_', 'event_type']).count().reset_index()

In [51]:
df.head()

,new_user_id,vacancy_id_,event_type
0,000089d26cdd49d68839c68bc10f2cf2,177380,preview_click_vacancy
1,000089d26cdd49d68839c68bc10f2cf2,177380,show_vacancy
2,000089d26cdd49d68839c68bc10f2cf2,181353,preview_click_vacancy
3,000089d26cdd49d68839c68bc10f2cf2,181353,show_vacancy
4,000089d26cdd49d68839c68bc10f2cf2,195890,preview_click_vacancy


In [52]:
df.info() # мы видим что базовый датасет уже меньше уже не 12+ млн а 9+

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9346647 entries, 0 to 9346646
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   new_user_id  object
 1   vacancy_id_  int64 
 2   event_type   object
dtypes: int64(1), object(2)
memory usage: 213.9+ MB


 ### Обработка данных первая исследовательская позиция (Маштабирование взаимодействий и их силы)
 ---

In [27]:
# Вот тут мы посмотрим на количество определенного вида взаимодействий в данных
df['event_type'].value_counts()

show_vacancy              4284563
preview_click_vacancy     3930718
click_response             354035
click_contacts             197767
preview_click_response     177402
click_favorite             149319
preview_click_favorite     100158
preview_click_contacts      78492
click_phone                 61251
preview_click_phone         12942
Name: event_type, dtype: int64

* 1 show_vacancy - просмотр вакансии
* 2 preview_click_vacancy - клик по карточке вакансии
* **3 click_response - отклик со страницы вакансии**
* **4 preview_click_response - отклик с карточки вакансии**
* 5 click_favorite - добавление вакансии в избранное  со страницы вакансии
* 6 preview_click_favorite - добавление вакансии в избраное с карточки вакансии
* **7 click_contacts - клик на контакты со страницы вакансии**
* **8 preview_click_contacts - клик на контакты из карточки вакансии**
* **9 click_phone - клик на номер телефона, указанный в вакансии**
* **10 preview_click_phone - клик на номер телефона из карточки вакансии**

Жирным выделены искомые ""откликнется"" и ""позвонит"""
! Стоит обратить внимание на то, что позиции 3 и 4 по частоте идут выше чем позиции 5 и 6, которые нам в положительное взаимодействие не засчитываются. В свзи с этим необходимо проработать несколько вариантов маштабирования данных

In [53]:
# проверим если закодировать значимость взаимодейстивий по их количеству в выборке - при прочих стандартных параметрах этот
# способ показал результат 0,041 - без каких либо ухищерений
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_response': 3.0,
                                     'click_contacts': 4.0,
                                     'preview_click_response': 5.0,
                                     'click_favorite': 6.0,
                                     'preview_click_favorite': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [54]:
# проверим правильно ли мы закодировали взаимодействия
df['eventStrength'].value_counts()

1.0     4284563
2.0     3930718
3.0      354035
4.0      197767
5.0      177402
6.0      149319
7.0      100158
8.0       78492
9.0       61251
10.0      12942
Name: eventStrength, dtype: int64

Далее несколько ячеек кода для различных вариаций оценки взаимодействия их стоит прогнать с базовыми значениями и посмотреть на результаты (возможны и другие предложения):

In [ ]:
# проверить потом - вот такая модель должна дать нам больший перекос в сторону искомых взаимодействий перед обычными
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 10.0,
                                     'preview_click_response': 10.0,
                                     'click_contacts': 10.0,
                                     'preview_click_contacts': 10.0,
                                     'click_phone': 10.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# Вот это разбиение по нашему заданию, то есть первые 4 отрицательные взаимодействия остальные положительные
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 2.0,
                                     'click_favorite': 3.0,
                                     'preview_click_favorite': 4.0,
                                     'click_response': 5.0,
                                     'click_contacts': 6.0,
                                     'preview_click_response': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [ ]:
# Вот это разбиение по нашему заданию, но незначимые мы закодируем нулями нужные единицами, но не знаю как это пойдет
# Нули в матрице могут восприниматься как отсутсвие взаимодействия, а это неправда
df['eventStrength'] = df['event_type'].map({'show_vacancy': 0.0,
                                     'preview_click_vacancy': 0.0,
                                     'click_favorite': 0.0,
                                     'preview_click_favorite': 0.0,
                                     'click_response': 1.0,
                                     'click_contacts': 1.0,
                                     'preview_click_response': 1.0,
                                     'preview_click_contacts': 1.0,
                                     'click_phone': 1.0,
                                     'preview_click_phone': 1.0})

In [ ]:
# Вот это разбиение по нашему заданию, но незначимые мы закодируем 1 нужные 2
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 2.0,
                                     'click_contacts': 2.0,
                                     'preview_click_response': 2.0,
                                     'preview_click_contacts': 2.0,
                                     'click_phone': 2.0,
                                     'preview_click_phone': 2.0})

In [ ]:
# Вот это разбиение по нашему заданию, ну и последний вариант это вот так:
df['eventStrength'] = df['event_type'].map({'show_vacancy': 0.1,
                                     'preview_click_vacancy': 0.1,
                                     'click_favorite': 0.1,
                                     'preview_click_favorite': 0.1,
                                     'click_response': 1.0,
                                     'click_contacts': 1.0,
                                     'preview_click_response': 1.0,
                                     'preview_click_contacts': 1.0,
                                     'click_phone': 1.0,
                                     'preview_click_phone': 1.0})

Группировка данных:

Еще один point № 3 по которому стоит провести исследование. Группировка схлопывает взаимодействия и выдает результат в виде силы взаимодействия

---

Тут мы поменяем на новый столбец соедиения устройства и пользователя

In [55]:
# сгруппируем сочетания ваканции и человека при этом суммируем историю из взаимодействия в числах, чем больще число
# тем больше интерес человека к вакансии - а значит больше и сила взаимодействия

grouped_df = df.groupby(['new_user_id', 'vacancy_id_']).sum('numeric_only').reset_index()

In [ ]:
# попробуем еще 2 варината группировки - по максимуму

#grouped_df = df.groupby(['new_user_id', 'vacancy_id_']).max('numeric_only').reset_index()

In [ ]:
# попробуем по пробуем группировку по среднему

# grouped_df = df.groupby(['new_user_id', 'vacancy_id_']).mean('numeric_only').reset_index()

In [56]:
# с 9 000 000 + наши данные схлопнулись до 4 500 000 +
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4663118 entries, 0 to 4663117
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   new_user_id    object 
 1   vacancy_id_    int64  
 2   eventStrength  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 106.7+ MB


In [57]:
# посмотрим как они выглядят теперь, далее можно будет сравнить как выглядит одно и тоже сравнение в контексте
# других вариантов маштабирования силы взаимодействий 
grouped_df.sample(10)

,new_user_id,vacancy_id_,eventStrength
3780133,cf46e9b6691d4e0480fbcdf457a985a0,114328,3.0
207446,0b0d7c1548f04693a5e89f2548561ece,168572,10.0
3881827,d4e478fff9bc499fa1fb265a4fcbbf1f,204285,3.0
1861438,65fd5b1b3ee8453f8212f78c26f4776d,190030,3.0
3687438,ca2cd9daa19646198ebe0868b8482335,113707,6.0
562815,1edd893b889447c08cc218e024cf4b84,135106,3.0
3355361,b81b22b2e4354ac4879a2ad65c7a3b90,168935,3.0
3563115,c375868270844d77a7b6475bf4b7c3ea,180526,3.0
856808,2ee7918fae3049a795218e52c8f03eaf,204287,3.0
3300004,b5223f45ed714a67abec880531e41628,188337,3.0


In [58]:
# посмотрим на одну вакансию в ключе ее взаимодействия с пользователями
grouped_df[grouped_df['vacancy_id_'] == 210068].sort_values('eventStrength', ascending=False).head(10)

,new_user_id,vacancy_id_,eventStrength
4252943,e9647deb68d6489c8f23fd7ff250f18c,210068,16.0
1577784,563d528309fd42399e1a97db5d42e9bb,210068,11.0
4657009,ffac1d4fb8ad4bb3a625d05f037a7f7f,210068,7.0
3216977,b08259ccb54a4e9b8fe4c5bf2dc0d180,210068,7.0
3046358,a73d2217a5a84960a43c5b422294e9a3,210068,6.0
1567378,55b03f3b497d416fa749e9c0832f7e07,210068,6.0
3822483,d1a5a18ca6f745d0820d6380b8b3b160,210068,3.0
2271992,7caf209c68c44e8b8e9cd8a318f3d17f,210068,3.0
2593743,8e529c2149044ec6aeb033d9665ee3f2,210068,3.0
2666833,9237ba25eea046a4a4114091e0c46bf2,210068,3.0


In [59]:
# посмотрим на одного пользователя в ключе его взаимодействия с вакансиями
grouped_df[grouped_df['new_user_id'] == 'e9647deb68d6489c8f23fd7ff250f18c'].sort_values('eventStrength', ascending=False).head(10)

,new_user_id,vacancy_id_,eventStrength
4252939,e9647deb68d6489c8f23fd7ff250f18c,193422,31.0
4252941,e9647deb68d6489c8f23fd7ff250f18c,199261,16.0
4252943,e9647deb68d6489c8f23fd7ff250f18c,210068,16.0
4252937,e9647deb68d6489c8f23fd7ff250f18c,189354,15.0
4252947,e9647deb68d6489c8f23fd7ff250f18c,233960,13.0
4252945,e9647deb68d6489c8f23fd7ff250f18c,224451,13.0
4252942,e9647deb68d6489c8f23fd7ff250f18c,203869,13.0
4252950,e9647deb68d6489c8f23fd7ff250f18c,236079,13.0
4252936,e9647deb68d6489c8f23fd7ff250f18c,181329,13.0
4252938,e9647deb68d6489c8f23fd7ff250f18c,192038,4.0


Вот к этому блоку нужно подойти очень внимательно - это выделение категориальных и числовых столбцов, в первых прогонах мы обошлись вот таким вариантом, но можно будет посмотреть и выделение нумированных вакансий (у нас в матрице с 0 до 100 000 значений нет, а номера есть, но может этого и ничего страшного)

---

In [60]:
grouped_df['new_user_id'] = grouped_df['new_user_id'].astype("category") # наименование вакансии в категорию
#grouped_df['vacancy_id_'] = grouped_df['vacancy_id_'].astype("category") # наименование вакансии в категорию

In [61]:
grouped_df['new_user_id_num'] = grouped_df['new_user_id'].cat.codes # вот тут выделяем числовые значения
#grouped_df['vacancy_id_num'] = grouped_df['vacancy_id_'].cat.codes # и вот тут выделим

Продолжаем готовить данные для разряженных матриц

---

In [62]:
grouped_df = grouped_df[['new_user_id', 'new_user_id_num', 'vacancy_id_', 'eventStrength']]

In [63]:
# В этом варианте кода мы пробуем запускать в матрицу номера вакансий которые нам дали изначально, что бы с ними потом не мучатся
matrix_data = grouped_df[['new_user_id_num', 'vacancy_id_', 'eventStrength']]

Вот тут стоит подумать о нормализации данных и вообще нужна ли она? Это еще один исследовательский блок давайте пробовавать разную нормализацию

In [ ]:
matrix_data['eventStrength_norm'] = (matrix_data['eventStrength'] - matrix_data['eventStrength'].mean()) / matrix_data['eventStrength'].std()

In [ ]:
matrix_data.head() # мне кажется минусы нам не нужны будут проверю другую

,cookie_id_num,vacancy_id_,eventStrength,eventStrength_norm
0,0,137659,3.0,-0.319517
1,0,153975,3.0,-0.319517
2,0,174953,9.0,0.771015
3,0,176171,3.0,-0.319517
4,0,182445,6.0,0.225749


In [ ]:
matrix_data['eventStrength_norm'] = (matrix_data['eventStrength'] - matrix_data['eventStrength'].min()) / (matrix_data['eventStrength'].max() - matrix_data['eventStrength'].min())

In [64]:
matrix_data.head() # мне кажется минусы нам не нужны будут проверю другую
# Вот этот датафрейм пойдет в матрицы (в следующий прогон загоним с нормализацией)

,new_user_id_num,vacancy_id_,eventStrength
0,0,177380,3.0
1,0,181353,3.0
2,0,195890,3.0
3,0,205562,3.0
4,0,211064,3.0


In [65]:
# а вот это остается нашим базовым информационным датафреймом
grouped_df.head(10)

,new_user_id,new_user_id_num,vacancy_id_,eventStrength
0,000089d26cdd49d68839c68bc10f2cf2,0,177380,3.0
1,000089d26cdd49d68839c68bc10f2cf2,0,181353,3.0
2,000089d26cdd49d68839c68bc10f2cf2,0,195890,3.0
3,000089d26cdd49d68839c68bc10f2cf2,0,205562,3.0
4,000089d26cdd49d68839c68bc10f2cf2,0,211064,3.0
5,000089d26cdd49d68839c68bc10f2cf2,0,258950,3.0
6,00008b2a006648b0b7eb409bbba203f1,1,108242,3.0
7,00008b2a006648b0b7eb409bbba203f1,1,110421,3.0
8,00008b2a006648b0b7eb409bbba203f1,1,113305,3.0
9,00008b2a006648b0b7eb409bbba203f1,1,120851,3.0


Вот в этом блоке мы должны соотнести наш цифровой номер cookie со строчным, для вывода ответов

---

Выбрали вариант работает на словарях

In [66]:
# кодируем числами все наши cookie_id
codes = grouped_df.new_user_id.astype('category')
# создаем словарик в котором ключ это код а значение текстовое поле cookie_id
cookie_dic = dict(enumerate(codes.cat.categories))
print (cookie_dic[1])

00008b2a006648b0b7eb409bbba203f1


In [67]:
# и создадим обратный словарик
cookie_to_code = dict(zip(cookie_dic.values(), cookie_dic.keys()))

In [68]:
print(cookie_to_code['00008b2a006648b0b7eb409bbba203f1'])

1


Создание разряженных матриц

---

In [69]:

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((matrix_data['eventStrength'].astype(float), (matrix_data['vacancy_id_'], matrix_data['new_user_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((matrix_data['eventStrength'].astype(float), (matrix_data['new_user_id_num'], matrix_data['vacancy_id_'])))

In [ ]:
# предыдущий код но с нормализацией

# item/user spare matrix
sparse_item_user = sparse.csr_matrix((data['eventStrength_norm'].astype(float), (data['vacancy_id_'], data['cookie_id_num'])))
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((data['eventStrength_norm'].astype(float), (data['cookie_id_num'], data['vacancy_id_'])))

In [70]:
# проверим размерности
sparse_item_user.shape
# стоит обратить внимание на то что размерность вакансий 260 168 - это потому что первых 100000 вакансий нет
# если бы мы кодировали - то размерность бы была 160 000 что то такое

(260168, 323030)

In [71]:
# и тут проверим
sparse_user_item.shape

(323030, 260168)

In [72]:
# ну и посмотрим что там внутри
print(sparse_user_item[0])

  (0, 177380)	3.0
  (0, 181353)	3.0
  (0, 195890)	3.0
  (0, 205562)	3.0
  (0, 211064)	3.0
  (0, 258950)	3.0


In [73]:
# и тут тоже посмотрим
print(sparse_item_user[200001])

  (0, 10616)	3.0
  (0, 36500)	3.0
  (0, 57867)	3.0
  (0, 58410)	9.0
  (0, 66839)	7.0
  (0, 77642)	3.0
  (0, 80748)	1.0
  (0, 270955)	3.0
  (0, 293568)	7.0


# Обучение моделей
Вот это третий исследовательский этап, мы должны много! очень много экспериментировать с:
 - factors = ?,
 - regularization = ?,
 - iterations = ?

---

In [74]:
# Инициируем наши модели 
model_item = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

In [75]:
model_user = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

Четвертый исследовательский блок связан с параметром альфа - он закомментирован в коде ниже. В тех статьях что я успел прочитать это чуть ли не главнй парметр, непонятно почему конечно, возможно он увеличивает и акцентирует отличие силе взаимодействия?, но тогда стоит задуматься нужна ли нам вообще нормализация? (в статьях встречались значения альфа в 15 и 50, надо пробовать)

Обучаем первую модель вакансия-пользователь (но мы ее скорее всего не будем использовать)

In [ ]:
# Вот эти параметры я подсмотрел, они не обязательные но наверное их лучше потом настраивать
#alpha = 50
#item_user_data = (sparse_item_user * alpha).astype('double')

item_user_data = sparse_item_user
# обучим модель на разряженной матрице  item/user/confidence weights - по русски вакансия/пользователь/веса заинтересованности
model_item.fit(item_user_data)

Обучаем первую модель пользователь - вакансия (вот ее в основном и принимают наши функции которые мы будем использовать)

In [76]:
# Вот эти параметры я подсмотрел, они не обязательные но наверное их лучше потом настраивать
#alpha = 50
#user_item_data = (sparse_user_item * alpha).astype('double')

user_item_data = sparse_user_item
# train the model on a sparse matrix of user/item confidence weights
model_user.fit(user_item_data)

  0%|          | 0/50 [00:00<?, ?it/s]

# Пробуем извечь пользу. То есть переходим к предсказаниям

---
(ниже собраны базовые функции библиотеки implicit)

## Создаем рекомендации для человека

---

Функция : recommend( )

Рекомендует элементы для пользователя
Вычисляет N лучших рекомендаций для пользователя и возвращает список идентификаторов элементов, оценка.

Параметры:

- userid ( int ) — идентификатор пользователя для расчета рекомендаций для
- user_items ( csr_matrix ) — разреженная матрица формы (number_users, number_items). Это позволяет нам искать понравившиеся предметы и их веса для пользователя. Это используется для фильтрации элементов, которые уже понравились, из выходных данных, а также для потенциального вычисления лучших элементов для этого пользователя.
- N ( int , необязательный ) — количество возвращаемых результатов
- filter_already_liked_items ( bool , необязательный ) — если установлено значение true, не возвращать элементы, присутствующие в обучающем наборе, которые были оценены указанным пользователем.
- filter_items ( последовательность целых чисел , необязательный ) — список дополнительных идентификаторов элементов, которые нужно отфильтровать из вывода.
- recalculate_user ( bool , необязательный ) — когда true, не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items

Возвращает:	
Список кортежей (itemid, score)

Тип возврата:	
список

Получим список рекомендаций для одного пользователя

---

In [ ]:
# модель тренируется на sparse matrix of user/item/confidence weights - выполнено
# model.fit(user_item_data) - выполнено

# порекомендуем вакансии для одного пользователя из тех что он еще не видел
userid = 0
recommendations = model_user.recommend(userid, user_item_data[userid], N = 5 , filter_already_liked_items = True)
print(recommendations) # уже отсортировано

(array([116823, 182870, 169194, 180382, 207423], dtype=int32), array([0.46656507, 0.41793132, 0.37318325, 0.3208992 , 0.30675507],
      dtype=float32))


In [ ]:
# или так мы можем сразу превращать их в список
userid = 0
ids, scores = model_user.recommend(userid, user_item_data[userid], N=5, filter_already_liked_items=True)
print(list(ids))
print(scores)

[116823, 182870, 169194, 180382, 207423]
[0.46656507 0.41793132 0.37318325 0.3208992  0.30675507]


In [ ]:
# порекомендуем вакансии для одного пользователя из тех что он уже взаимодействовал
userid = 0
recommendations = model_user.recommend(userid, user_item_data[userid], N = 5, filter_already_liked_items = False)
print(recommendations) # уже отсортировано

(array([174953, 116823, 182870, 169194, 180382], dtype=int32), array([0.6994151 , 0.46656507, 0.41793132, 0.37318325, 0.3208992 ],
      dtype=float32))


Получим списко рекомендаций для списка пользователей

---

In [ ]:
# Такой код позволяет получать рекомендации для списка пользователей сразу!!!
userids = np.arange(1000) # вот тут у нас список пользователей
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)
ids, ids.shape

(array([[116823, 182870, 169194, 180382, 207423],
        [217683, 260154, 163708, 115589, 210229],
        [110792, 148714, 258441, 113707, 244077],
        ...,
        [230322, 230326, 127887, 133932, 182879],
        [253946, 253817, 208761, 122122, 104528],
        [164602, 260154, 110421, 116823, 164481]], dtype=int32),
 (1000, 5))

In [ ]:
userids = np.arange(1000) # вот тут у нас список пользователей
group_recommendations = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)


In [ ]:
# Вот тут нужно написать код который записывает данные в датафрейм
result = pd.DataFrame({'пользователи' : userids, 'рекомендации' : list(ids)})

In [ ]:
result.head()

,пользователи,рекомендации
0,0,"[116823, 182870, 169194, 180382, 207423]"
1,1,"[217683, 260154, 163708, 115589, 210229]"
2,2,"[110792, 148714, 258441, 113707, 244077]"
3,3,"[242145, 153469, 126489, 197991, 207356]"
4,4,"[113707, 153245, 148714, 149024, 116823]"


## Найдем похожие вакансии на вакансию №

---

Функция: similar_items()

Вычисляет список похожих элементов

Параметры:	
- itemid (int) – идентификатор строки элемента, для которого извлекаются похожие элементы
- N (int, необязательно) – количество похожих элементов для возврата.
- react_users (csr_matrix, необязательно) – разреженная матрица формы (number_items, number_users). Это позволяет нам просмотреть откликнувшихся пользователей и их веса для элемента.
- recallculate_item (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние элемента и вместо этого пересчитывайте из переданного в react_users

ВОЗВРАТ:	
Список кортежей (itemid, оценка)

Возвращаемый тип:	
Список

In [ ]:
# Найти похожую вакансию на вакансию №
itemid = 100005
related = model_user.similar_items(itemid)
print(list(related[0])) # стоит обратить внимание на то, что первой выходит сама эта вакансия
print(list(related[0][1:]))

[100005, 188746, 221652, 189497, 180201, 201215, 181059, 223080, 143875, 189711]
[188746, 221652, 189497, 180201, 201215, 181059, 223080, 143875, 189711]


## Найдем похожего пользователя

Функция: similar_users( )

Вычисляет список похожих пользователей

Параметры:

- userid ( int ) — идентификатор строки пользователя для получения похожих пользователей для
- N ( int , необязательный ) — количество похожих пользователей, которых необходимо вернуть.

Возвращает:	
Список кортежей (userid, score)

Тип возврата:	
список

In [ ]:
userid = 100005
related_users = model_user.similar_users(userid)
print(list(related_users[0])) # стоит обратить внимание на то, что первой выходит этот самый пользователь
print(list(related_users[0][1:]))

[100005, 113260, 192092, 143338, 307425, 281362, 81256, 58263, 131196, 66705]
[113260, 192092, 143338, 307425, 281362, 81256, 58263, 131196, 66705]


## Ранжируем элементы для пользователя

---

Функция: rank_items()

Ранжирует заданные элементы для пользователя и возвращает отсортированный список элементов.

Параметры:

- userid (int) – идентификатор пользователя для вычисления рекомендаций
- user_items (csr_matrix) – разреженная матрица формы (number_users, number_items). Это позволяет нам (необязательно) пересчитывать пользовательские факторы (см. параметр reconculate_user) по мере необходимости
- selected_items (List of itemids) – (список идентификаторов элементов)
- recalculate_user (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items



Возвращаемый тип:	
Список

In [ ]:
userid = 100005
rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004])
print(rank_elements) 


(array([100002, 100001, 100004, 100003]), array([4.0011927e-03, 4.6750274e-04, 3.4725224e-05, 1.9556945e-05],
      dtype=float32))


<ipython-input-82-0aa5fe56e486>:2: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004])


# Переходим к испытаниям на нашей ситуации

---

Загружаем данные

In [77]:
# Вот это данные от партнера, стоит отметить что там есть строки с 2 вакансиями и более 5
test = pd.read_parquet('test_public_mfti.parquet', engine='pyarrow')

In [78]:
test.head()

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"


In [79]:
test['new_user_id'] = test['cookie_id'].map(new_user_id)

In [80]:
# получаем для них столбец с номерными вариантами user как мы это закодировали ранее
test['new_user_id_num'] = test['new_user_id'].map(cookie_to_code)

In [81]:
test.head()

,cookie_id,vacancy_id_,new_user_id,new_user_id_num
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",477e719bfbcc4071a8acc1b357492b00,90572
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",753a2a268ae84ab49475e62736c02860,148017
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",d01c76b282364fbb8195f326de3e893c,262750
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",6adb4495090b454395d5aa2f1a5ff9d9,135026
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]",b107696e3ff54e8a83182828bb845bc6,223617


In [82]:
# проверим перевод в коды на правильность
print(new_user_id['0034bc7f404341ba8412665453e7825a'])
print(new_user_id['000cd76cd33f43d4a1ac1d16d10f8bf7'])
print(new_user_id['01de50c280794cec8804f16f45f847b7'])

753a2a268ae84ab49475e62736c02860
477e719bfbcc4071a8acc1b357492b00
b107696e3ff54e8a83182828bb845bc6


In [83]:
# проверим перевод в коды на правильность
print(cookie_to_code['753a2a268ae84ab49475e62736c02860'])
print(cookie_to_code['d01c76b282364fbb8195f326de3e893c'])
print(cookie_to_code['b107696e3ff54e8a83182828bb845bc6'])

148017
262750
223617


In [84]:
# получаем из датафрейма список пользователей, которых мы загрузим в модель
test_list = list(test['new_user_id_num'])
test_list_str = list(test['cookie_id'])
print(test_list[:5])
print(test_list_str[:5])

[90572, 148017, 262750, 135026, 223617]
['000cd76cd33f43d4a1ac1d16d10f8bf7', '0034bc7f404341ba8412665453e7825a', '00a6c5a64a274c55a836402bdeb3b2c4', '015937a125b14e74bdff1cddc49f9172', '01de50c280794cec8804f16f45f847b7']


In [85]:
# Ячейка поличения предсказаний, загружаем в модель полученный список кодов пользоватлей 
userids = test_list
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)

In [86]:
# а вот тут мы делаем хитро в первую колонку записываем строковые имена пользователей, а в список рекомендаций полученные предсказания
test_prediction = pd.DataFrame({'cookie_id' : test_list_str, 'top_recommendations' : list(ids)})

In [87]:
# посмотрим что получилось (выглядит красиво как и требуется)
test_prediction.head()

,cookie_id,top_recommendations
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[260154, 146630, 185913, 117525, 181275]"
1,0034bc7f404341ba8412665453e7825a,"[106943, 103881, 138634, 171332, 207423]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[227708, 109079, 113305, 148714, 242642]"
3,015937a125b14e74bdff1cddc49f9172,"[114583, 193331, 205606, 164588, 192565]"
4,01de50c280794cec8804f16f45f847b7,"[128278, 137702, 117532, 253817, 253946]"


In [88]:
# вот эта наша функция получения метрики
def precision_n(predictions, df_test_list_top, k=5):
  #recommendations num:
    n_rec_k = len(df_test_list_top)*k
  #true reccomendations:
    matches=0
    for row in range(len(predictions)):
      row_match = set(predictions['top_recommendations'][row]).intersection(set(df_test_list_top['vacancy_id_'][row]))  
      matches += len(row_match)
  #Results:
    precisions = matches / n_rec_k if n_rec_k != 0 else 0
    return precisions

In [89]:
# Ну и собственно вот результат
precision_n(test_prediction, test)

0.039119170984455956